# Community Detection Analysis for Knowledge Graph

This notebook performs comprehensive community detection analysis on a relational knowledge graph.

## Contents
1. Data Loading and Graph Construction
2. Graph Statistics and Exploration
3. Community Detection Algorithms
4. Quality Metrics Evaluation
5. Comparative Analysis
6. Visualizations
7. Results Export

---

## 1. Import Libraries

In [1]:
# Core libraries
import networkx as nx
from networkx.algorithms import community
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict, Counter
import json
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"NetworkX version: {nx.__version__}")

Libraries imported successfully!
NetworkX version: 3.6.1


## 2. Data Loading and Graph Construction

In [2]:
def load_knowledge_graph(filepath):
    """
    Load knowledge graph from triplet format (subject relation object)
    
    Args:
        filepath: Path to the knowledge graph file
    
    Returns:
        G: NetworkX graph
        edges_with_relations: List of tuples (subject, object, relation)
    """
    print("Loading knowledge graph...")
    G = nx.Graph()
    edges_with_relations = []
    
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                subject, relation, obj = parts[0], parts[1], parts[2]
                G.add_edge(subject, obj, relation=relation)
                edges_with_relations.append((subject, obj, relation))
    
    print(f"✓ Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G, edges_with_relations

# Load the graph
filepath = 'train.txt'  # Update this path as needed
G, edges = load_knowledge_graph(filepath)

Loading knowledge graph...
✓ Loaded graph with 1316 nodes and 7480 edges


## 3. Graph Statistics and Exploration

In [ ]:
def print_graph_statistics(G):
    """
    Print comprehensive graph statistics
    """
    print("=" * 60)
    print("KNOWLEDGE GRAPH STATISTICS")
    print("=" * 60)
    
    print(f"\nBasic Statistics:")
    print(f"  Nodes: {G.number_of_nodes():,}")
    print(f"  Edges: {G.number_of_edges():,}")
    print(f"  Density: {nx.density(G):.6f}")
    print(f"  Average Degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
    print(f"  Average Clustering Coefficient: {nx.average_clustering(G):.4f}")
    
    print(f"\nConnectivity:")
    print(f"  Is Connected: {nx.is_connected(G)}")
    
    if nx.is_connected(G):
        print(f"  Diameter: {nx.diameter(G)}")
        print(f"  Average Shortest Path Length: {nx.average_shortest_path_length(G):.2f}")
    else:
        num_components = nx.number_connected_components(G)
        print(f"  Number of Connected Components: {num_components}")
        largest_cc = max(nx.connected_components(G), key=len)
        print(f"  Largest Component Size: {len(largest_cc)} nodes ({len(largest_cc)/G.number_of_nodes()*100:.1f}%)")
    
    print("=" * 60)

print_graph_statistics(G)

In [ ]:
# Analyze relation types
relation_counts = Counter()
for _, _, data in G.edges(data=True):
    relation_counts[data.get('relation', 'unknown')] += 1

print("Top 10 Relation Types:")
for relation, count in relation_counts.most_common(10):
    print(f"  {relation}: {count}")

## 4. Quality Metrics Functions

In [ ]:
def compute_modularity(G, communities):
    """Compute modularity of community partition"""
    return community.modularity(G, communities)

def compute_coverage(G, communities):
    """Compute coverage - fraction of edges within communities"""
    from networkx.algorithms.community.quality import intra_community_edges
    total_edges = G.number_of_edges()
    if total_edges == 0:
        return 0.0
    intra_edges = intra_community_edges(G, communities)
    return intra_edges / total_edges

def compute_performance(G, communities):
    """Compute performance metric - fraction of node pairs correctly classified"""
    n = G.number_of_nodes()
    if n <= 1:
        return 1.0
    
    # Create community membership dict
    node_to_comm = {}
    for i, comm in enumerate(communities):
        for node in comm:
            node_to_comm[node] = i
    
    # For large graphs, sample pairs
    max_pairs = 10000
    nodes = list(G.nodes())
    total_possible = n * (n - 1) // 2
    
    if total_possible <= max_pairs:
        correct = 0
        total = 0
        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)):
                total += 1
                u, v = nodes[i], nodes[j]
                same_community = node_to_comm.get(u) == node_to_comm.get(v)
                has_edge = G.has_edge(u, v)
                if (same_community and has_edge) or (not same_community and not has_edge):
                    correct += 1
    else:
        import random
        random.seed(42)
        correct = 0
        total = max_pairs
        for _ in range(max_pairs):
            u, v = random.sample(nodes, 2)
            same_community = node_to_comm.get(u) == node_to_comm.get(v)
            has_edge = G.has_edge(u, v)
            if (same_community and has_edge) or (not same_community and not has_edge):
                correct += 1
    
    return correct / total if total > 0 else 0.0

def compute_community_diameter(G, communities):
    """Compute diameter for each community"""
    diameters = []
    for comm in communities:
        subgraph = G.subgraph(comm)
        if nx.is_connected(subgraph) and len(comm) > 1:
            diameters.append(nx.diameter(subgraph))
        else:
            diameters.append(0)
    return diameters

def compute_community_density(G, communities):
    """Compute density for each community"""
    densities = []
    for comm in communities:
        subgraph = G.subgraph(comm)
        densities.append(nx.density(subgraph))
    return densities

def compute_clustering_coefficient(G, communities):
    """Compute average clustering coefficient for each community"""
    clustering_coeffs = []
    for comm in communities:
        subgraph = G.subgraph(comm)
        if len(comm) > 1:
            clustering_coeffs.append(nx.average_clustering(subgraph))
        else:
            clustering_coeffs.append(0.0)
    return clustering_coeffs

def compute_conductance(G, communities):
    """Compute conductance for each community"""
    conductances = []
    for comm in communities:
        if len(comm) > 0:
            internal_edges = G.subgraph(comm).number_of_edges()
            cut_edges = nx.cuts.cut_size(G, comm)
            vol_s = 2 * internal_edges + cut_edges
            vol_total = 2 * G.number_of_edges()
            vol_complement = vol_total - vol_s
            
            if vol_s > 0 and vol_complement > 0:
                conductance = cut_edges / min(vol_s, vol_complement)
            else:
                conductance = 0
            conductances.append(conductance)
        else:
            conductances.append(0)
    return conductances

print("✓ Quality metric functions defined")

## 5. Community Detection Algorithms

### 5.1 Louvain Algorithm

In [ ]:
print("\n" + "="*60)
print("LOUVAIN ALGORITHM")
print("="*60)

# Run Louvain
communities_louvain = community.louvain_communities(G, seed=42)

# Compute metrics
louvain_results = {
    'algorithm': 'Louvain',
    'communities': communities_louvain,
    'num_communities': len(communities_louvain),
    'sizes': [len(c) for c in communities_louvain],
    'modularity': compute_modularity(G, communities_louvain),
    'coverage': compute_coverage(G, communities_louvain),
    'performance': compute_performance(G, communities_louvain),
    'diameters': compute_community_diameter(G, communities_louvain),
    'densities': compute_community_density(G, communities_louvain),
    'clustering': compute_clustering_coefficient(G, communities_louvain),
    'conductances': compute_conductance(G, communities_louvain)
}

print(f"\nNumber of communities: {louvain_results['num_communities']}")
print(f"Modularity: {louvain_results['modularity']:.4f}")
print(f"Coverage: {louvain_results['coverage']:.4f}")
print(f"Performance: {louvain_results['performance']:.4f}")
print(f"Avg community size: {np.mean(louvain_results['sizes']):.2f}")
print(f"Avg diameter: {np.mean([d for d in louvain_results['diameters'] if d > 0]):.2f}")
print(f"Avg density: {np.mean(louvain_results['densities']):.4f}")

### 5.2 Label Propagation Algorithm

In [ ]:
print("\n" + "="*60)
print("LABEL PROPAGATION ALGORITHM")
print("="*60)

# Run Label Propagation
communities_lp = list(community.label_propagation_communities(G))

# Compute metrics
lp_results = {
    'algorithm': 'Label Propagation',
    'communities': communities_lp,
    'num_communities': len(communities_lp),
    'sizes': [len(c) for c in communities_lp],
    'modularity': compute_modularity(G, communities_lp),
    'coverage': compute_coverage(G, communities_lp),
    'performance': compute_performance(G, communities_lp),
    'diameters': compute_community_diameter(G, communities_lp),
    'densities': compute_community_density(G, communities_lp),
    'clustering': compute_clustering_coefficient(G, communities_lp),
    'conductances': compute_conductance(G, communities_lp)
}

print(f"\nNumber of communities: {lp_results['num_communities']}")
print(f"Modularity: {lp_results['modularity']:.4f}")
print(f"Coverage: {lp_results['coverage']:.4f}")
print(f"Performance: {lp_results['performance']:.4f}")
print(f"Avg community size: {np.mean(lp_results['sizes']):.2f}")
print(f"Avg diameter: {np.mean([d for d in lp_results['diameters'] if d > 0]):.2f}")
print(f"Avg density: {np.mean(lp_results['densities']):.4f}")

### 5.3 Greedy Modularity Optimization

In [ ]:
print("\n" + "="*60)
print("GREEDY MODULARITY OPTIMIZATION")
print("="*60)

# Run Greedy Modularity
communities_greedy = list(community.greedy_modularity_communities(G))

# Compute metrics
greedy_results = {
    'algorithm': 'Greedy Modularity',
    'communities': communities_greedy,
    'num_communities': len(communities_greedy),
    'sizes': [len(c) for c in communities_greedy],
    'modularity': compute_modularity(G, communities_greedy),
    'coverage': compute_coverage(G, communities_greedy),
    'performance': compute_performance(G, communities_greedy),
    'diameters': compute_community_diameter(G, communities_greedy),
    'densities': compute_community_density(G, communities_greedy),
    'clustering': compute_clustering_coefficient(G, communities_greedy),
    'conductances': compute_conductance(G, communities_greedy)
}

print(f"\nNumber of communities: {greedy_results['num_communities']}")
print(f"Modularity: {greedy_results['modularity']:.4f}")
print(f"Coverage: {greedy_results['coverage']:.4f}")
print(f"Performance: {greedy_results['performance']:.4f}")
print(f"Avg community size: {np.mean(greedy_results['sizes']):.2f}")
print(f"Avg diameter: {np.mean([d for d in greedy_results['diameters'] if d > 0]):.2f}")
print(f"Avg density: {np.mean(greedy_results['densities']):.4f}")

### 5.4 Girvan-Newman Algorithm (Optional - Slow for large graphs)

In [ ]:
# Only run on smaller graphs (< 200 nodes)
if G.number_of_nodes() < 200:
    print("\n" + "="*60)
    print("GIRVAN-NEWMAN ALGORITHM")
    print("="*60)
    
    # Run Girvan-Newman
    comp = community.girvan_newman(G)
    
    # Find optimal communities based on modularity
    best_modularity = -1
    best_communities = None
    
    for i, communities_gn in enumerate(comp):
        if i > 20:  # Limit iterations
            break
        mod = compute_modularity(G, communities_gn)
        if mod > best_modularity:
            best_modularity = mod
            best_communities = communities_gn
    
    communities_gn = list(best_communities)
    
    # Compute metrics
    gn_results = {
        'algorithm': 'Girvan-Newman',
        'communities': communities_gn,
        'num_communities': len(communities_gn),
        'sizes': [len(c) for c in communities_gn],
        'modularity': compute_modularity(G, communities_gn),
        'coverage': compute_coverage(G, communities_gn),
        'performance': compute_performance(G, communities_gn),
        'diameters': compute_community_diameter(G, communities_gn),
        'densities': compute_community_density(G, communities_gn),
        'clustering': compute_clustering_coefficient(G, communities_gn),
        'conductances': compute_conductance(G, communities_gn)
    }
    
    print(f"\nNumber of communities: {gn_results['num_communities']}")
    print(f"Modularity: {gn_results['modularity']:.4f}")
    print(f"Coverage: {gn_results['coverage']:.4f}")
    print(f"Performance: {gn_results['performance']:.4f}")
else:
    print("\nSkipping Girvan-Newman (graph too large)")
    gn_results = None

## 6. Comparative Analysis

In [ ]:
# Collect all results
all_results = [louvain_results, lp_results, greedy_results]
if gn_results:
    all_results.append(gn_results)

# Create comparison dataframe
comparison_data = []
for r in all_results:
    comparison_data.append({
        'Algorithm': r['algorithm'],
        'Communities': r['num_communities'],
        'Modularity': f"{r['modularity']:.4f}",
        'Coverage': f"{r['coverage']:.4f}",
        'Performance': f"{r['performance']:.4f}",
        'Avg Size': f"{np.mean(r['sizes']):.1f}",
        'Avg Density': f"{np.mean(r['densities']):.4f}",
        'Avg Clustering': f"{np.mean(r['clustering']):.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("COMPARATIVE ANALYSIS")
print("="*80)
print(comparison_df.to_string(index=False))

# Find best algorithm for each metric
print("\n" + "="*80)
print("BEST ALGORITHM BY METRIC")
print("="*80)

best_modularity = max(all_results, key=lambda x: x['modularity'])
best_coverage = max(all_results, key=lambda x: x['coverage'])
best_performance = max(all_results, key=lambda x: x['performance'])

print(f"Best Modularity: {best_modularity['algorithm']} ({best_modularity['modularity']:.4f})")
print(f"Best Coverage: {best_coverage['algorithm']} ({best_coverage['coverage']:.4f})")
print(f"Best Performance: {best_performance['algorithm']} ({best_performance['performance']:.4f})")

## 7. Visualizations

### 7.1 Quality Metrics Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Community Detection Quality Metrics Comparison', fontsize=16, fontweight='bold')

algorithms = [r['algorithm'] for r in all_results]

# Modularity
modularity = [r['modularity'] for r in all_results]
axes[0, 0].bar(algorithms, modularity, color='steelblue', alpha=0.7)
axes[0, 0].set_ylabel('Modularity', fontweight='bold')
axes[0, 0].set_title('Modularity (Higher is Better)')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)

# Coverage
coverage = [r['coverage'] for r in all_results]
axes[0, 1].bar(algorithms, coverage, color='coral', alpha=0.7)
axes[0, 1].set_ylabel('Coverage', fontweight='bold')
axes[0, 1].set_title('Coverage (Higher is Better)')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# Performance
performance = [r['performance'] for r in all_results]
axes[1, 0].bar(algorithms, performance, color='mediumseagreen', alpha=0.7)
axes[1, 0].set_ylabel('Performance', fontweight='bold')
axes[1, 0].set_title('Performance (Higher is Better)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# Number of communities
num_comms = [r['num_communities'] for r in all_results]
axes[1, 1].bar(algorithms, num_comms, color='mediumpurple', alpha=0.7)
axes[1, 1].set_ylabel('Number of Communities', fontweight='bold')
axes[1, 1].set_title('Number of Communities Detected')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 7.2 Community Size Distributions

In [ ]:
n_algos = len(all_results)
n_cols = 2
n_rows = (n_algos + 1) // 2

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 5*n_rows))
if n_algos == 1:
    axes = [axes]
else:
    axes = axes.flatten()

fig.suptitle('Community Size Distributions', fontsize=16, fontweight='bold')

for idx, result in enumerate(all_results):
    sizes = result['sizes']
    axes[idx].hist(sizes, bins=min(30, len(set(sizes))), color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel('Community Size', fontweight='bold')
    axes[idx].set_ylabel('Frequency', fontweight='bold')
    axes[idx].set_title(f"{result['algorithm']}\n(Avg: {np.mean(sizes):.1f}, Median: {np.median(sizes):.1f})")
    axes[idx].grid(axis='y', alpha=0.3)

# Hide unused subplots
for idx in range(n_algos, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

### 7.3 Density vs Diameter

In [ ]:
fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 5*n_rows))
if n_algos == 1:
    axes = [axes]
else:
    axes = axes.flatten()

fig.suptitle('Community Density vs Diameter', fontsize=16, fontweight='bold')

for idx, result in enumerate(all_results):
    densities = result['densities']
    diameters = [d if d > 0 else 0 for d in result['diameters']]
    axes[idx].scatter(densities, diameters, alpha=0.6, s=50)
    axes[idx].set_xlabel('Density', fontweight='bold')
    axes[idx].set_ylabel('Diameter', fontweight='bold')
    axes[idx].set_title(result['algorithm'])
    axes[idx].grid(alpha=0.3)

# Hide unused subplots
for idx in range(n_algos, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

### 7.4 Network Visualization

In [ ]:
# Visualize using best algorithm (Louvain)
communities_viz = communities_louvain

# Create community color mapping
node_colors = {}
color_map = plt.cm.tab20.colors + plt.cm.tab20b.colors + plt.cm.tab20c.colors

for idx, comm in enumerate(communities_viz):
    color = color_map[idx % len(color_map)]
    for node in comm:
        node_colors[node] = color

# Get largest connected component for visualization
if not nx.is_connected(G):
    largest_cc = max(nx.connected_components(G), key=len)
    G_viz = G.subgraph(largest_cc).copy()
    print(f"Visualizing largest component with {len(largest_cc)} nodes")
else:
    G_viz = G

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Spring layout
pos = nx.spring_layout(G_viz, seed=42, k=0.5, iterations=50)
node_color_list = [node_colors.get(node, 'gray') for node in G_viz.nodes()]

nx.draw_networkx_nodes(G_viz, pos, node_color=node_color_list, node_size=100, alpha=0.8, ax=axes[0])
nx.draw_networkx_edges(G_viz, pos, alpha=0.2, width=0.5, ax=axes[0])
axes[0].set_title('Spring Layout (Colored by Community)', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Circular layout by community
communities_viz_filtered = []
for comm in communities_viz:
    comm_in_viz = [n for n in comm if n in G_viz]
    if comm_in_viz:
        communities_viz_filtered.append(comm_in_viz)

pos_circular = {}
angle_step = 2 * np.pi / len(communities_viz_filtered)

for idx, comm in enumerate(communities_viz_filtered):
    angle = idx * angle_step
    radius = 1.0
    comm_angle_step = 2 * np.pi / len(comm) if len(comm) > 1 else 0
    for node_idx, node in enumerate(comm):
        node_angle = angle + node_idx * comm_angle_step * 0.3
        x = radius * np.cos(angle) + 0.3 * np.cos(node_angle)
        y = radius * np.sin(angle) + 0.3 * np.sin(node_angle)
        pos_circular[node] = np.array([x, y])

nx.draw_networkx_nodes(G_viz, pos_circular, node_color=node_color_list, node_size=100, alpha=0.8, ax=axes[1])
nx.draw_networkx_edges(G_viz, pos_circular, alpha=0.2, width=0.5, ax=axes[1])
axes[1].set_title('Circular Layout by Community', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

### 7.5 Degree Distribution

In [ ]:
degrees = [d for n, d in G.degree()]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(degrees, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Degree', fontweight='bold')
axes[0].set_ylabel('Frequency', fontweight='bold')
axes[0].set_title('Degree Distribution', fontweight='bold')
axes[0].grid(alpha=0.3)

# Log-log plot
degree_count = Counter(degrees)
x = sorted(degree_count.keys())
y = [degree_count[k] for k in x]

axes[1].loglog(x, y, 'o-', color='coral', markersize=6, alpha=0.7)
axes[1].set_xlabel('Degree (log scale)', fontweight='bold')
axes[1].set_ylabel('Frequency (log scale)', fontweight='bold')
axes[1].set_title('Degree Distribution (Log-Log)', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 7.6 Relation Types Analysis

In [ ]:
# Get top 15 relations
top_relations = relation_counts.most_common(15)
relations, counts = zip(*top_relations)

fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(range(len(relations)), counts, color='mediumseagreen', alpha=0.7)
ax.set_yticks(range(len(relations)))
ax.set_yticklabels(relations)
ax.set_xlabel('Frequency', fontweight='bold')
ax.set_title('Top 15 Relation Types in Knowledge Graph', fontweight='bold', fontsize=14)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Export Results

In [ ]:
# Save summary CSV
comparison_df.to_csv('community_detection_summary.csv', index=False)
print("✓ Saved: community_detection_summary.csv")

# Save detailed per-community metrics for each algorithm
for result in all_results:
    algo_name = result['algorithm'].replace(' ', '_').lower()
    comm_df = pd.DataFrame({
        'community_id': range(len(result['sizes'])),
        'size': result['sizes'],
        'diameter': result['diameters'],
        'density': result['densities'],
        'clustering': result['clustering'],
        'conductance': result['conductances']
    })
    filename = f'communities_{algo_name}.csv'
    comm_df.to_csv(filename, index=False)
    print(f"✓ Saved: {filename}")

print("\n✓ All results exported successfully!")

## 9. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

print(f"\nGraph Statistics:")
print(f"  Total Nodes: {G.number_of_nodes():,}")
print(f"  Total Edges: {G.number_of_edges():,}")
print(f"  Graph Density: {nx.density(G):.6f}")
print(f"  Average Clustering: {nx.average_clustering(G):.4f}")

print(f"\nBest Algorithm Overall: {best_modularity['algorithm']}")
print(f"  Communities Detected: {best_modularity['num_communities']}")
print(f"  Modularity: {best_modularity['modularity']:.4f}")
print(f"  Coverage: {best_modularity['coverage']:.4f}")
print(f"  Performance: {best_modularity['performance']:.4f}")

print("\n" + "="*80)
print("Key Insights:")
print("="*80)

if best_modularity['modularity'] > 0.7:
    print("✓ Excellent community structure detected (Modularity > 0.7)")
elif best_modularity['modularity'] > 0.3:
    print("✓ Good community structure detected (Modularity > 0.3)")
else:
    print("⚠ Weak community structure (Modularity < 0.3)")

if best_coverage['coverage'] > 0.9:
    print("✓ Very high coverage - most edges within communities")
elif best_coverage['coverage'] > 0.7:
    print("✓ Good coverage - majority of edges within communities")

avg_conductance = np.mean(best_modularity['conductances'])
if avg_conductance < 0.1:
    print("✓ Low conductance - well-separated communities")
elif avg_conductance < 0.3:
    print("✓ Moderate conductance - reasonably separated communities")

print("\n" + "="*80)
print("Analysis Complete!")
print("="*80)